In [ ]:
import os
import json
import time
import numpy as np
from qiskit import transpile
from disqco.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from disqco.circuits.cp_fraction import cp_fraction
from disqco.parti.FM.FM_main import *
from disqco.parti.FM.multilevel_FM import *
import math as mt

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_FM_fixed_levels_stoch.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_FM_fixed_levels_stoch.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 97, 8)

for i, num_qubits in enumerate(sizes):
    # For each increase of 8 qubits, increase the number of partitions by 1
    num_partitions = 2 + i

    # Create an All-to-All network
    qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
    
    # Sweep the fraction parameter from 0.1 to 0.9
    for fraction in np.arange(0.3, 0.9, 0.2):
        
        # Collect data for computing means across 10 iterations
        iteration_data = []
        
        for iteration in range(5):

            
            
            # -------------------------
            # 1. Define/redefine circuit
            # -------------------------
            circuit = cp_fraction(num_qubits, num_qubits, fraction)
            base_graph = QuantumCircuitHyperGraph(circuit, group_gates=True, anti_diag=True)
            depth = base_graph.depth
            num_levels = np.ceil(mt.log2(depth))
            initial_assignment = set_initial_partitions(qpu_info,num_qubits, depth ,num_partitions, reduced=True)
            
            # -------------------------
            # 2. Fine-grained partitioning
            # -------------------------
            graph_list = [base_graph]
            mapping_list = [{i : set([i]) for i in range(depth)}]

            assignment_list_f, cost_list_f, time_list_f = multilevel_FM(graph_list,
                                                                  mapping_list,
                                                                  initial_assignment,
                                                                    qpu_info,
                                                                    limit = num_qubits*0.125,
                                                                    pass_list= [5*num_levels],
                                                                    stochastic=True,
                                                                    lock_nodes=False,
                                                                    log = False,
                                                                    add_initial = False,
                                                                    costs = None)
            total_time_f = sum(time_list_f)
            min_cost_f = min(cost_list_f)
            # -------------------------
            # 3. Window-based refinement
            # -------------------------

            assignment_list_w, cost_list_w, time_list_w = MLFM_window(base_graph, 
                num_levels=num_levels,
                initial_assignment=initial_assignment,  
                qpu_info= qpu_info, 
                limit = num_qubits*0.125, 
                pass_list= [5]*(num_levels+2), 
                stochastic=True, 
                lock_nodes=False,
                log = False,
                add_initial = False,
                costs = None)
            
            total_time_w = sum(time_list_w)
            min_cost_w = min(cost_list_w)

            
            # -------------------------
            # 4. Block refinement
            # -------------------------
            assignment_list_b, cost_list_b, time_list_b = MLFM_blocks(base_graph,
                                            num_levels=num_levels,
                                            initial_assignment=initial_assignment,  
                                            qpu_info= qpu_info, 
                                            limit = num_qubits*0.125, 
                                            pass_list= [5]*(num_levels+2), 
                                            stochastic=True, 
                                            lock_nodes=False,
                                            log = False,
                                            add_initial = False,
                                            costs = None,
                                            full = False)
            
            total_time_b = sum(time_list_b)
            min_cost_b = min(cost_list_b)
            
            # -------------------------
            # 5. Recursive refinement
            # -------------------------
            assignment_list_r, cost_list_r, time_list_r = MLFM_recursive(base_graph,
                                        initial_assignment,  
                                        qpu_info, 
                                        limit = num_qubits*0.125, 
                                        pass_list= [5]*(num_levels+2), 
                                        stochastic=True, 
                                        lock_nodes=False,
                                        log = False,
                                        add_initial = False,
                                        costs = None)
            
            total_time_r = sum(time_list_r)
            min_cost_r = min(cost_list_r)

            
            # -------------------------
            # 6. Store iteration-level results
            # -------------------------
            result_entry = {
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "fraction": float(fraction),
                "iteration": iteration,
                "f_cost": min_cost_f,
                "w_cost": min_cost_w,
                "b_cost": min_cost_b,
                "r_cost":  min_cost_r,
                "time_f": total_time_f,
                "time_w": total_time_w,
                "time_b": total_time_b,
                "time_r": total_time_r,
            }
            
            detailed_results.append(result_entry)
            iteration_data.append(result_entry)
            
            # Update detailed JSON right away
            with open(detailed_filename, "w") as f:
                json.dump(detailed_results, f, indent=2)
        
        # ---------------------------------------------------------------------
        # After 10 iterations, compute the means and log them
        # ---------------------------------------------------------------------
        f_cost_list = [x["f_cost"] for x in iteration_data]
        w_cost_list = [x["w_cost"] for x in iteration_data]
        b_cost_list = [x["b_cost"] for x in iteration_data]
        r_cost_list = [x["r_cost"] for x in iteration_data]
        
        f_time_list = [x["time_f"] for x in iteration_data]
        w_time_list = [x["time_w"] for x in iteration_data]
        b_time_list = [x["time_b"] for x in iteration_data]
        r_time_list = [x["time_r"] for x in iteration_data]
        
        mean_f_cost = float(np.mean(f_cost_list))
        mean_w_cost = float(np.mean(w_cost_list))
        mean_b_cost = float(np.mean(b_cost_list))
        mean_r_cost = float(np.mean(r_cost_list))
        
        mean_f_time = float(np.mean(f_time_list))
        mean_w_time = float(np.mean(w_time_list))
        mean_b_time = float(np.mean(b_time_list))
        mean_r_time = float(np.mean(r_time_list))
        
        # Print to console for quick logging
        print("=============================================")
        print(f"Finished 10 iterations for:")
        print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}, fraction={fraction}")
        print("Mean Costs:")
        print(f"  `Fine`:    {mean_f_cost:.3f}")
        print(f"  Window: {mean_w_cost:.3f}")
        print(f"  Block: {mean_b_cost:.3f}")
        print(f"  Recursive:{mean_r_cost:.3f}")
        print("Mean Times (s):")
        print(f"  Fine:    {mean_f_time:.3f}")
        print(f"  Window: {mean_w_time:.3f}")
        print(f"  Block: {mean_b_time:.3f}")
        print(f"  Recursive:{mean_r_time:.3f}")
        print("=============================================")
        
        # Store the aggregated means in a separate JSON
        mean_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "fraction": float(fraction),
            "mean_f_cost": mean_f_cost,
            "mean_w_cost": mean_w_cost,
            "mean_b_cost": mean_b_cost,
            "mean_r_cost": mean_r_cost,
            "mean_f_time": mean_f_time,
            "mean_w_time": mean_w_time,
            "mean_b_time": mean_b_time,
            "mean_r_time": mean_r_time,
        }
        
        mean_results.append(mean_entry)
        
        # Update the means JSON file
        with open(means_filename, "w") as f:
            json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Finished 10 iterations for:
  # Qubits: 16, # Partitions: 2, fraction=0.3
Mean Costs:
  `Fine`:    13.000
  Window: 6.400
  Block: 10.400
  Recursive:5.400
Mean Times (s):
  Fine:    0.030
  Window: 0.022
  Block: 0.021
  Recursive:0.033
Finished 10 iterations for:
  # Qubits: 16, # Partitions: 2, fraction=0.5
Mean Costs:
  `Fine`:    22.800
  Window: 13.000
  Block: 19.000
  Recursive:13.400
Mean Times (s):
  Fine:    0.032
  Window: 0.024
  Block: 0.022
  Recursive:0.035
Finished 10 iterations for:
  # Qubits: 16, # Partitions: 2, fraction=0.7
Mean Costs:
  `Fine`:    21.800
  Window: 16.200
  Block: 19.400
  Recursive:15.800
Mean Times (s):
  Fine:    0.037
  Window: 0.028
  Block: 0.025
  Recursive:0.039
Finished 10 iterations for:
  # Qubits: 16, # Partitions: 2, fraction=0.9
Mean Costs:
  `Fine`:    18.600
  Window: 14.200
  Block: 16.800
  Recursive:13.600
Mean Times (s):
  Fine:    0.045
  Window: 0.035
  Block: 0.030
  Recursive:0.044
Finished 10 iterations for:
  # Qubits: 24

In [ ]:
import os
import json
import time
import numpy as np
from qiskit import transpile
from disqco.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from disqco.circuits.cp_fraction import cp_fraction
from disqco.parti.FM.FM_main import *
from disqco.parti.FM.multilevel_FM import *
import math as mt

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_FM_fine_vs_ML.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_FM_fine_vs_ML.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 65, 8)

num_partitions_list = [2,3,4,5,6,7,8]

for i, num_qubits in enumerate(sizes):
    # For each increase of 8 qubits, increase the number of partitions by 1
        num_partitions = num_partitions_list[i]

        # Create an All-to-All network
        qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
        
        # Sweep the fraction parameter from 0.1 to 0.9
        for fraction in np.arange(0.5, 0.6, 0.2):
            
            # Collect data for computing means across 10 iterations
            iteration_data = []

            
            for iteration in range(5):
                print(f"Running iteration {iteration} for {num_qubits} qubits, {num_partitions} partitions, fraction={fraction}")
                
                # -------------------------
                # 1. Define/redefine circuit
                # -------------------------
                circuit = cp_fraction(num_qubits, num_qubits, fraction)
                base_graph = QuantumCircuitHyperGraph(circuit, group_gates=True, anti_diag=True)
                depth = base_graph.depth
                num_levels = int(mt.log2(depth))
                initial_assignment = set_initial_partitions(qpu_info,num_qubits, depth ,num_partitions, reduced=True)
                
                # -------------------------
                # 2. Fine-grained partitioning
                # -------------------------
                graph_list = [base_graph]
                mapping_list = [{i : set([i]) for i in range(depth)}]

                assignment_list_f, cost_list_f, time_list_f = multilevel_FM(graph_list,
                                                                    mapping_list,
                                                                    initial_assignment,
                                                                        qpu_info,
                                                                        limit = num_qubits * depth * 0.125,
                                                                        pass_list= [5*(num_levels+2)],
                                                                        stochastic=True,
                                                                        lock_nodes=False,
                                                                        log = False,
                                                                        add_initial = False,
                                                                        costs = None)
                total_time_f = sum(time_list_f)
                min_cost_f = min(cost_list_f)
                # -------------------------
                # 3. Window-based refinement
                # -------------------------

                assignment_list_w, cost_list_w, time_list_w = MLFM_window(base_graph, 
                    num_levels=num_levels,
                    initial_assignment=initial_assignment,  
                    qpu_info= qpu_info, 
                    limit = num_qubits,
                    pass_list= [5]*(num_levels+2), 
                    stochastic=False, 
                    lock_nodes=False,
                    log = False,
                    add_initial = False,
                    costs = None)
                
                total_time_w = sum(time_list_w)
                min_cost_w = min(cost_list_w)

                
                # -------------------------
                # 4. Block refinement
                # -------------------------
                assignment_list_b, cost_list_b, time_list_b = MLFM_blocks(base_graph,
                                                num_levels=num_levels+1,
                                                initial_assignment=initial_assignment,  
                                                qpu_info = qpu_info, 
                                                limit = num_qubits , 
                                                pass_list= [5]*(num_levels+2), 
                                                stochastic=True, 
                                                lock_nodes=False,
                                                log = False,
                                                add_initial = False,
                                                costs = None)
                
                total_time_b = sum(time_list_b)
                min_cost_b = min(cost_list_b)
                
                # -------------------------
                # 5. Recursive refinement
                # -------------------------
                assignment_list_r, cost_list_r, time_list_r = MLFM_recursive(base_graph,
                                            initial_assignment,  
                                            qpu_info, 
                                            limit = num_qubits , 
                                            pass_list= [5]*(num_levels+2), 
                                            stochastic=True, 
                                            lock_nodes=False,
                                            log = False,
                                            add_initial = False,
                                            costs = None)
                
                total_time_r = sum(time_list_r)
                min_cost_r = min(cost_list_r)

                
                # -------------------------
                # 6. Store iteration-level results
                # -------------------------
                result_entry = {
                    "num_qubits": num_qubits,
                    "num_partitions": num_partitions,
                    "fraction": float(fraction),
                    "iteration": iteration,
                    "f_cost": min_cost_f,
                    "w_cost": min_cost_w,
                    "b_cost": min_cost_b,
                    "r_cost":  min_cost_r,
                    "time_f": total_time_f,
                    "time_w": total_time_w,
                    "time_b": total_time_b,
                    "time_r": total_time_r,
                }
                
                detailed_results.append(result_entry)
                iteration_data.append(result_entry)
                
                # Update detailed JSON right away
                with open(detailed_filename, "w") as f:
                    json.dump(detailed_results, f, indent=2)
            
            # ---------------------------------------------------------------------
            # After 10 iterations, compute the means and log them
            # ---------------------------------------------------------------------
            f_cost_list = [x["f_cost"] for x in iteration_data]
            w_cost_list = [x["w_cost"] for x in iteration_data]
            b_cost_list = [x["b_cost"] for x in iteration_data]
            r_cost_list = [x["r_cost"] for x in iteration_data]
            
            f_time_list = [x["time_f"] for x in iteration_data]
            w_time_list = [x["time_w"] for x in iteration_data]
            b_time_list = [x["time_b"] for x in iteration_data]
            r_time_list = [x["time_r"] for x in iteration_data]
            
            mean_f_cost = float(np.mean(f_cost_list))
            mean_w_cost = float(np.mean(w_cost_list))
            mean_b_cost = float(np.mean(b_cost_list))
            mean_r_cost = float(np.mean(r_cost_list))
            
            mean_f_time = float(np.mean(f_time_list))
            mean_w_time = float(np.mean(w_time_list))
            mean_b_time = float(np.mean(b_time_list))
            mean_r_time = float(np.mean(r_time_list))
            
            # Print to console for quick logging
            print("=============================================")
            print(f"Finished 10 iterations for:")
            print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}, fraction={fraction}")
            print("Mean Costs:")
            print(f"  `Fine`:    {mean_f_cost:.3f}")
            print(f"  Window: {mean_w_cost:.3f}")
            print(f"  Block: {mean_b_cost:.3f}")
            print(f"  Recursive:{mean_r_cost:.3f}")
            print("Mean Times (s):")
            print(f"  Fine:    {mean_f_time:.3f}")
            print(f"  Window: {mean_w_time:.3f}")
            print(f"  Block: {mean_b_time:.3f}")
            print(f"  Recursive:{mean_r_time:.3f}")
            print("=============================================")
            
            # Store the aggregated means in a separate JSON
            mean_entry = {
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "fraction": float(fraction),
                "mean_f_cost": mean_f_cost,
                "mean_w_cost": mean_w_cost,
                "mean_b_cost": mean_b_cost,
                "mean_r_cost": mean_r_cost,
                "mean_f_time": mean_f_time,
                "mean_w_time": mean_w_time,
                "mean_b_time": mean_b_time,
                "mean_r_time": mean_r_time,
            }
            
            mean_results.append(mean_entry)
            
            # Update the means JSON file
            with open(means_filename, "w") as f:
                json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Running iteration 0 for 16 qubits, 2 partitions, fraction=0.5
Number of hypergraphs coarsened: 5
Running iteration 1 for 16 qubits, 2 partitions, fraction=0.5
Number of hypergraphs coarsened: 5
Running iteration 2 for 16 qubits, 2 partitions, fraction=0.5
Number of hypergraphs coarsened: 5
Running iteration 3 for 16 qubits, 2 partitions, fraction=0.5
Number of hypergraphs coarsened: 5
Running iteration 4 for 16 qubits, 2 partitions, fraction=0.5
Number of hypergraphs coarsened: 5
Finished 10 iterations for:
  # Qubits: 16, # Partitions: 2, fraction=0.5
Mean Costs:
  `Fine`:    13.400
  Window: 11.600
  Block: 11.400
  Recursive:10.800
Mean Times (s):
  Fine:    0.090
  Window: 0.052
  Block: 0.062
  Recursive:0.064
Running iteration 0 for 24 qubits, 3 partitions, fraction=0.5
Number of hypergraphs coarsened: 6
Running iteration 1 for 24 qubits, 3 partitions, fraction=0.5
Number of hypergraphs coarsened: 6
Running iteration 2 for 24 qubits, 3 partitions, fraction=0.5
Number of hypergrap

Created dat_files/fraction_0.3_cost.dat and dat_files/fraction_0.3_time.dat
Created dat_files/fraction_0.5_cost.dat and dat_files/fraction_0.5_time.dat
Created dat_files/fraction_0.7_cost.dat and dat_files/fraction_0.7_time.dat
Created dat_files/fraction_0.9_cost.dat and dat_files/fraction_0.9_time.dat
Done generating .dat files by fraction.


Saved figure: fraction_0.3_cost.png
Saved figure: fraction_0.5_cost.png
Saved figure: fraction_0.7_cost.png
Saved figure: fraction_0.9_cost.png
Saved figure: fraction_0.3_time.png
Saved figure: fraction_0.5_time.png
Saved figure: fraction_0.7_time.png
Saved figure: fraction_0.9_time.png
